PyTorch Hubを利用します
* https://pytorch.org/docs/stable/hub.html
* https://pytorch.org/hub/

今回使うのはNVIDIAが公開しているSSDのモデルです
* https://pytorch.org/hub/nvidia_deeplearningexamples_ssd/

サンプルコードはBSD-3-Clauseライセンスで公開されている以下のサンプルを参照しています。  
https://nvidia.github.io/Torch-TensorRT/_notebooks/ssd-object-detection-demo.html  
`Copyright (c) 2020-present, NVIDIA CORPORATION. All rights reserved.`  
https://github.com/NVIDIA/Torch-TensorRT/blob/master/LICENSE

In [3]:
import torch
print(torch.__version__)

1.11.0
False


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")  
else:
    device = torch.device("cpu")
print(device)

cpu


In [6]:
# COCOデータセットでトレーニングしたSSDモデルをダウンロードする
# UserWarning: pytorch_quantization module not found, quantization will not be available とワーニングが出る場合があるがひとまず無視して問題ない
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', map_location=torch.device('cpu'))
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

Using cache found in /home/ubuntu/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# GPUを有効にしてモデルを読み込む
ssd_model.to('cuda')
ssd_model.eval()

In [ ]:
# 結果可視化用にCOCOのアノテーションラベルリストを取得
classes_to_labels = utils.get_coco_object_dictionary()

# 結果可視化用関数定義
from matplotlib import pyplot as plt
import matplotlib.patches as patches

# The utility plots the images and predicted bounding boxes (with confidence scores).
def plot_results(best_results):
    for image_idx in range(len(best_results)):
        fig, ax = plt.subplots(1)
        # Show original, denormalized image...
        image = inputs[image_idx] / 2 + 0.5
        ax.imshow(image)
        # ...with detections
        bboxes, classes, confidences = best_results[image_idx]
        for idx in range(len(bboxes)):
            left, bot, right, top = bboxes[idx]
            x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

In [ ]:
# 使用する画像リストを作成（ローカルにある画像へのパスを指定しても可）
uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg',
    'http://images.cocodataset.org/val2017/000000037777.jpg',
    'http://images.cocodataset.org/val2017/000000252219.jpg',
    'https://rt-net.jp/wp-content/uploads/2020/10/edu_seminar_image_raspimouse.png-600x400.jpg',
    'https://rt-net.jp/mobility/wp-content/uploads/2022/03/s-IMG_2553_b.jpg',
    'https://ultralytics.com/images/zidane.jpg'
]

In [ ]:
# 画像取得しネットワークへ入力できるように変換
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs)

In [ ]:
# 物体検出
with torch.no_grad():
    detections_batch = ssd_model(tensor)

results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]

In [ ]:
# 結果可視化
plot_results(best_results_per_input)